In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim # slim is a wrapper that makes building networks easier
import matplotlib.pyplot as plot
from matplotlib import animation
from matplotlib.patches import Rectangle, Circle
from collections import deque 
from IPython.display import display, HTML

# Linear problem introduction

We'll start with a very simple problem: a linear track with 6 different locations. At each location, the agent will be able to take the action "left," which will move it to the position to the left, or "right" which will move it to the right. (If it is at the left end of the track, the "left" action will just leave the agent in the same position.) The agent will start at the left-most end of the track, and we'll give it a reward of +1 for reaching the end of track, which we'll consider a terminal state. 

<img src="linear.png">

Clearly the optimal policy is for the agent to move "Right" until it reaches the end of the track. In this part of the homework, we'll explore learning this task with a tabular Q-learning system.

## Mathematical questions:

(There are 16 questions across 5 sections on this homework, some with code chunks interspersed, make sure you answer all of them! Please answer the questions in a separate document. We have provided one in .docx format for your convenience, but if you prefer another format you may use your own.)

1\. Assuming an optimal, completely greedy policy, and a discount factor of gamma = 0.9, calculate the Q-value of each (state, action) pair. 

2\. Under the same assumptions, calculate the value of every state (this shouldn't be much work given the last part).

## Linear problem, random, and tabular Q controller implementations

Note: The code for the problems and controllers in this homework is separated out into modules, to avoid cluttering the document. However, we encourage you to take a look at the code if you're curious or want to make sure you understand what is going on!

In [ ]:
import linear

## Linear problem questions

To answer these questions, run the code chunk below, which prints the Q tables at logarthimically spaced steps in the learning process. Make sure to restart your Jupyter Kernel and run all preceding code chunks (Cell->Run all above) before you start.

3\. About how long (how many training episodes) does it take the tabular Q-system to converge to the optimal Q values you calculated above?

4\. For which states do the Q-values converge earlier? For which actions? Why? 

5\. How does changing epsilon affect this? (Try editing the code chunk below to set epsilon = 0.2)

In [ ]:
test_episode_spacing = [16, 64, 256, 1024, 4096]

lp = linear.linear_problem()

# create a tabular Q controller 
np.random.seed(1)
tq = linear.tabular_Q_controller(epsilon=0.05) # you can change epsilon here


tq.set_testing()
print("Initial:")
lp.run_trial(tq, testing=True)
print()

prev = 0
for i in range(len(test_episode_spacing)):
    this_cycle_episodes = test_episode_spacing[i] - prev
    prev = this_cycle_episodes
    
    tq.set_training()
    lp.run_k_trials(tq, this_cycle_episodes)
    tq.set_testing()
    print("After %i training episodes" % (test_episode_spacing[i]))
    lp.run_trial(tq, testing=True)
    print("Q-values:")
    tq.print_pretty_Q_table()
    print()


6\. Run the code chunk below, and observe that the random controller performs better than a randomly initialized tabular Q-learner (before learning). Why does this occur?

In [ ]:
for seed in range(5):
    # create a random controller and run a trial with it
    rc = linear.random_controller()
    np.random.seed(seed)
    print("Random")
    lp.run_trial(rc, testing=True)

    # create a tabular Q controller and run a trial with it,
    # then run 10000 training trials and run another testing trial
    np.random.seed(seed)
    tq = linear.tabular_Q_controller()
    tq.set_testing()
    print("Tabular (pre-training)")
    lp.run_trial(tq, testing=True)
    print()

# Cartpole problem introduction

Now we'll explore something a little more interesting: the cartpole problem:

<img src="cartpole.png">

A pole is attached to a pivot on top of a cart which moves along a one-dimensional track. The goal of the task is to keep the pole balanced (standing upright) by moving the cart side to side. To make this into a MDP like we've discussed, we need the following elements:

* *Agent:* the controller of the cart
* *Environment:* the cart/world/physics
* *State:* we'll define the state to be a tuple of (x position of cart, x velocity of cart, angle of pole, angular velocity of pole).
* *Terminal states:* we'll end the episode when the pole tips too far over (> 15 degrees, in this implementation) or when the cart goes too far to either side (> 2.5 units).
* *Actions:* to keep it simple, we'll have only two actions: apply a force of +F toward the right, or -F toward the left, which we'll call "right" and "left," respectively.
* *Rewards:* To keep things simple and clear, we'll only give a reward in terminal states. Since all terminal states are losing, the reward will be -1.

We'll compare two Q-learning approaches to this task in this homework: 

* *Tabular:* "standard" Q-learning
* *DQN:* A deep-Q network that approximates the Q-function, loosely inspired by the Atari game playing paper.

We'll also compare to a baseline controller that takes random actions at every step.

Some of the code chunks in this part of the document have been run for you already since they take a non-trivial amount of time (especially the DQN training). However, we still encourage you to play around with the code and get your hands dirty! (Even the DQN training chunk should only take 10-15 minutes on a modern system.)

## Conceptual questions

7\. Since the reward for every *episode* (not every action!) will be -1, why would a Q-learning system learn any interesting behavior on this task?

8\. Why might a DQN (or some other function approximator) be an appropriate choice here? Compare this with the linear track problem, would a DQN be helpful there? 

Food for thought (no answer necessary): Can you create an MDP on which a function approximator could not provide a benefit over a tabular controller? In this case, interference would probably cause the function approximator to do much worse.

## Cartpole problem and random controller implementation

Run the code chunks below to try a few random controllers with different random seeds, to get a baseline for comparison, and animate one, to get an intuition for how the tasks looks. (Again, feel free to poke around in the associated module if you're curious.)

In [ ]:
import cartpole
cpp = cartpole.cartpole_problem()

In [ ]:
for i in range(10):
    np.random.seed(i)
    cpc = cartpole.random_controller()
    cpp.run_trial(cpc, testing=True)

In [ ]:
# and animate one!
cpp.run_trial(cpc, testing=True, animate=True)

Notice how the random controller quickly loses control of the pole and lets it tip over. (You can run the chunk a few times to see different trajectories for the random controller.)

## Tabular Q learning

There is a difficulty in making this a tabular Q-learning problem: it's not a finite MDP! Since the space of x values, angles, and velocities is continuous, it's actually uncountably infinite. In order to avoid trying to make an infinite table, we'll discretize the space (actually quite drastically), by chopping the position and angle dimensions into 3 bins , and the velocity dimensions into 5, thus reducing the continuous state space to 225 discrete states. It's not perfect by any stretch of the imagination, but as you'll see below, it offers quite an improvement over the random controller. 

In [ ]:
np.random.seed(0)
tqc = cartpole.tabular_Q_controller()
tqc.set_testing()
cpp.run_trial(tqc, testing=True)
# for trainable controllers, we'll run a few testing trials during
# training to see how they evolve
for step in range(5):
    tqc.set_training()
    cpp.run_k_trials(tqc, 1000)
    tqc.set_testing()
    cpp.run_trial(tqc, testing=True)
    
cpp.run_trial(tqc, testing=True, animate=True)

Notice how the tabular Q system gets the balance pretty well, but is unable to keep the car within bounds while keeping the pole balanced (it tries to remain in bounds toward the end, but then the pole tips over...)

## Tabular Q-learning questions

9\. The tabular Q-learning system does much better than a random controller, but it still only lives about 5 times as long. What could we do to improve the tabular Q system's performance on this task further? For whatever you propose, how would it affect training? 

10\. Try setting gamma = 0.0 (living in the moment), by running the next cell. What happens? Why?

In [ ]:
np.random.seed(0)
tqc = cartpole.tabular_Q_controller(gamma=0.)
tqc.set_testing()
cpp.run_trial(tqc, testing=True)
for i in range(5):
    tqc.set_training()
    cpp.run_k_trials(tqc, 1000)
    tqc.set_testing()
    cpp.run_trial(tqc, testing=True)
    
cpp.run_trial(tqc, testing=True, animate=True)

11\. Try setting gamma = 1 (living in all moments at once), by running the next cell. Naively, one might expect to get random behavior, since all trials get the same total reward, and gamma = 1 is essentially saying that the total reward is all that matters, not when the reward appears. However, this is not what actually happens. Why?

In [ ]:
np.random.seed(0)
tqc = cartpole.tabular_Q_controller(gamma=1.)
tqc.set_testing()
cpp.run_trial(tqc, testing=True)
for i in range(5):
    tqc.set_training()
    cpp.run_k_trials(tqc, 1000)
    tqc.set_testing()
    cpp.run_trial(tqc, testing=True)
    
cpp.run_trial(tqc, testing=True, animate=True)

12\. Try setting epsilon = 1 (random behavior while training) by running the next cell. What happens? Why?

In [ ]:
np.random.seed(0)
tqc = cartpole.tabular_Q_controller(epsilon=1.)
tqc.set_testing()
cpp.run_trial(tqc, testing=True)
for i in range(5):
    tqc.set_training()
    cpp.run_k_trials(tqc, 1000)
    tqc.set_testing()
    cpp.run_trial(tqc, testing=True)
    
cpp.run_trial(tqc, testing=True, animate=True)

13\. Try setting epsilon = 0 (no exploration), by running the next cell. Why does this happen here, and what might be different about other tasks that makes exploration important?

In [ ]:
np.random.seed(0)
tqc = cartpole.tabular_Q_controller(epsilon=0.)
tqc.set_testing()
cpp.run_trial(tqc, testing=True)
for i in range(5):
    tqc.set_training()
    cpp.run_k_trials(tqc, 1000)
    tqc.set_testing()
    cpp.run_trial(tqc, testing=True)
    
cpp.run_trial(tqc, testing=True, animate=True)

Food for thought (no answer necessary): Are the discretization values very important? (The current values were picked by a few quick rounds of trial and error.) If we discretized the space more finely, would we see better results? Is it better to space the breaks linearly or quadratically?

## DQN

In some ways, creating the DQN is simpler than creating the tabular Q-learning system. Neural nets can accept continuous input, so we can simply pass the current state to the network without discretizing. We implemented a simple DQN below, with two hidden layers (each with 100 hidden units and a Tanh non-linearity), before an output layer (two units, for the respective Q-values of "left" and "right," again with a Tanh non-linearity). We implemented a simple replay buffer that at each time step stores the current experience and samples one of the previous 1000 time steps to replay. (The buffer persists across episodes.) The system was trained using the Adam optimizer with a learning rate of 0.0001. By default the training is epsilon greedy, with epsilon = 0.05, and the testing is greedy. As for the tabular and random systems, the chosen action applies a force of a fixed pre-determined magnitude F either toward the right (+F), or toward the left (-F).

As you'll see below, this system does quite a bit better. In fact, it reaches the time limit at which the cartpole code stops by default (1000 steps). (However, note there is some cross-platform variation in the number of episodes required to reach this state...)

We have provided representative results from running the training of the DQN below, so that you don't need to run the training yourself (which will take about 15 minutes on a reasonably modern machine). We have provided a video of the system trained with the replay buffer -- run the code chunk  containing `HTML(...)` to see it. (If the video doesn't display in the notebook when you run the code chunk, you should be able to watch the raw .mp4 file which is also included with the homework.) 

In [ ]:
dqn = cartpole.dqn_controller(replay_buffer=True, rseed=0)
dqn.set_testing()
cpp.run_trial(dqn, testing=True)
for i in range(10):
    dqn.set_training()
    cpp.run_k_trials(dqn, 1000)
    dqn.set_testing()
    this_lifetime = cpp.run_trial(dqn, testing=True)
    if this_lifetime == 1000:
        break
    
cpp.run_trial(dqn, testing=True, animate=True)    
cpp.run_k_trials(dqn, 100) # run an extra hundred trials with 
                           # some randomness (due to exploration)
                           # and some learning to evaluate robustness


#### Results on my machine (so you don't have to run the training)
```
Ran testing trial with DQN Controller, achieved a lifetime of 24 steps
Ran 1000 trials with DQN Controller, (average lifetime of 18.629000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 23 steps
Ran 1000 trials with DQN Controller, (average lifetime of 19.294000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 16 steps
Ran 1000 trials with DQN Controller, (average lifetime of 19.562000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 19 steps
Ran 1000 trials with DQN Controller, (average lifetime of 19.773000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 18 steps
Ran 1000 trials with DQN Controller, (average lifetime of 38.124000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 44 steps
Ran 1000 trials with DQN Controller, (average lifetime of 133.583000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 155 steps
Ran 1000 trials with DQN Controller, (average lifetime of 138.172000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 109 steps
Ran 1000 trials with DQN Controller, (average lifetime of 295.301000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 1000 steps

Ran 100 trials with DQN Controller, (average lifetime of 1000.000000 steps)
```


In [ ]:
HTML("""<video controls src="dqn_cartpole.mp4" type="video/mp4" />""")

Notice how the DQN solves both problems: it is able to keep the pole balanced, and stop moving towards the left when it gets too close to the edge of the screen.

## DQN questions

14\. Why does the DQN take more episodes to train than the tabular Q-learning system? 

15\. In my implementation, I used the tanh activation function at the output layer. Why might this be an appropriate choice here? More specifically, what are some activation functions that would probably NOT yield good results at the output layer?

16\. What happens if we turn off the replay buffer? Why might it be important? (See the text below the code chunk for the output of a representative run on my machine.)

In [ ]:
dqn = cartpole.dqn_controller(replay_buffer=False, rseed=0)
dqn.set_testing()
cpp.run_trial(dqn, testing=True)
for i in range(10):
    dqn.set_training()
    cpp.run_k_trials(dqn, 1000)
    dqn.set_testing()
    this_lifetime = cpp.run_trial(dqn, testing=True)
    if this_lifetime == 1000:
        break
    
cpp.run_trial(dqn, testing=True, animate=True)

#### Results on my machine (so you don't have to run the training)
```
Ran testing trial with DQN Controller, achieved a lifetime of 21 steps
Ran 1000 trials with DQN Controller, (average lifetime of 18.985000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 16 steps
Ran 1000 trials with DQN Controller, (average lifetime of 19.454000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 21 steps
Ran 1000 trials with DQN Controller, (average lifetime of 19.749000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 17 steps
Ran 1000 trials with DQN Controller, (average lifetime of 18.036000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 21 steps
Ran 1000 trials with DQN Controller, (average lifetime of 21.084000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 21 steps
Ran 1000 trials with DQN Controller, (average lifetime of 40.156000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 30 steps
Ran 1000 trials with DQN Controller, (average lifetime of 113.236000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 156 steps
Ran 1000 trials with DQN Controller, (average lifetime of 96.128000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 91 steps
Ran 1000 trials with DQN Controller, (average lifetime of 95.368000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 121 steps
Ran 1000 trials with DQN Controller, (average lifetime of 110.364000 steps)
Ran testing trial with DQN Controller, achieved a lifetime of 110 steps
```


Food for thought (no answer necessary): If you train the DQN without replay for longer, can you get it to converge? Do you think this would scale to more complex tasks? If you gave the DQN the same discretized states that the tabular Q-network gets, would it do any better than the tabular system does? (Try it out if you're curious!)